In [1]:
import os
os.chdir('../')
%pwd

'/Users/martonszentpali/mlops'

In [2]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')
data.info()
data.describe()
data.isnull().sum()
data.duplicated().sum()
data.drop_duplicates(inplace=True)
data.shape


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/winequality-red.csv'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: Path
    unzip_data_dir: Path
    all_schema: dict

In [ ]:
from MLops.constants import *
from MLops.utils import *
from MLops.utils.common import read_yaml



In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config    


In [ ]:
import os
from MLops.utils import logger
from MLops.utils.common import *


In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema
            missing_cols = []
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e


In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-10-17 14:59:58,084: INFO: common: yaml file: configs/config.yaml loaded successfully]
[2025-10-17 14:59:58,085: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-17 14:59:58,086: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-17 14:59:58,086: INFO: common: created directory at: artifacts]
[2025-10-17 14:59:58,086: INFO: common: created directory at: artifacts/data_validation]
